In [58]:
import pandas as pd
import numpy as np

In [59]:
수익성 = pd.read_csv('./datasets/코스닥/코스닥 수익성.csv',encoding='cp949')
안정성 = pd.read_csv('./datasets/코스닥/코스닥 안정성.csv',encoding='cp949')
활동성 = pd.read_csv('./datasets/코스닥/코스닥 활동성.csv',encoding='cp949')
성장성 = pd.read_csv('./datasets/코스닥/코스닥 성장성 계산용.csv',encoding='cp949')
현금흐름 = pd.read_csv('./datasets/코스닥/코스닥 현금흐름 + 누적수익성비율.csv',encoding='cp949')
연구개발 = pd.read_csv('./datasets/코스닥/코스닥 rs.csv',encoding='cp949')
EBIT = pd.read_csv('./datasets/코스닥/에비~ 코스닥.csv',encoding='cp949')
per_pbr_총부채 = pd.read_csv('./datasets/코스닥/per pbr 총부채 코스닥.csv',encoding='cp949')
직원수_업력 = pd.read_csv('./datasets/코스닥/코스닥 직원수 급여 업력 외국인 대주주.csv',encoding='cp949')
시가총액 = pd.read_csv('./datasets/코스닥/코스닥시가총액.csv',index_col=0)  #### 시총도 붙여야됨
단기차입금 = pd.read_csv('./datasets/코스닥/코스닥 단기차입금 유동성 장기부채.csv',encoding='cp949')
거시변수 = pd.read_excel('./datasets/거시경제 변수/거시변수.xlsx', engine="openpyxl")
상장일 = pd.read_csv('./datasets/코스닥/코스닥 상장일.csv',encoding='cp949')
대분류 = pd.read_csv('./datasets/코스닥/코스닥 대분류.csv',encoding='cp949')
유형자산 = pd.read_csv("./datasets/코스닥/코스닥 유형자산증가율.csv", encoding= "cp949")

In [60]:
대분류.drop(['통계청 한국표준산업분류 코드 10차(대분류)'],axis=1,inplace=True)

In [61]:
df = 안정성.merge(수익성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(활동성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(성장성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(현금흐름,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(연구개발,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(EBIT,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(per_pbr_총부채,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(직원수_업력,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(단기차입금,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(상장일,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(대분류,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(유형자산,on=['회사명','거래소코드','회계년도'],how='left')

In [62]:
len(df.columns)

97

In [63]:
## 거래소코드 패딩
def padding(x):
    x = str(x)
    return x.rjust(6, "0")
## 외국계기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

# 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

def func_not(x):
    return not x

def change_year_2(x):
    return (x[:4])

In [64]:
## 거래소 코드 패딩
df['거래소코드'] = df['거래소코드'].map(padding)
# 회계년도 제거
df['회계년도'] = df['회계년도'].map(change_year)

In [65]:
df.isna().sum()

회사명                                               0
거래소코드                                             0
회계년도                                              0
자산(*)(요약)(IFRS)                                1844
자본(*)(요약)(IFRS)                                1844
차입금의존도(IFRS)(백만원)                              2051
유보율(IFRS)                                      2051
현금비율(IFRS)                                     2051
부채비율(IFRS)                                     2051
당좌비율(IFRS)                                     2051
순운전자본비율(IFRS)                                  2051
*이자비용(요약)(IFRS)(백만원)                           1844
* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)               1844
감가상각비(IFRS)(천원)                                1844
자산(*)(요약)(IFRS연결)                              5530
자본(*)(요약)(IFRS연결)                              5530
차입금의존도(IFRS연결)                                 5611
유보율(IFRS연결)(백만원)                               5611
현금비율(IFRS연결)(백만원)                              5611
부채비율(IFRS연결)

In [66]:
## 중복되는 데이터 제거
df.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)

## 없음


In [67]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 15617 개
전체 기업 수 1744


In [68]:
## 외국계 기업 제거
print("제거할 데이터 수 : ", len(df[df["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
print('제거할 외국계 기업 수',len(df[~df["거래소코드"].map(func_isnt_fcompany)].회사명.unique()))

df = df[df["거래소코드"].map(func_isnt_fcompany)]
print('남은 데이터 수',df.shape[0]) # 외국계기업 데이터 24개 제거
print('남은 기업 수',len(df['회사명'].unique()))

제거할 데이터 수 :  152
제거할 외국계 기업 수 23
남은 데이터 수 15465
남은 기업 수 1721


In [69]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df["회사명"].unique()))
print("거래소코드수 : ", len(df["거래소코드"].unique()))

회사수 :  1721
거래소코드수 :  1743


In [70]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df["회사명"].unique()):
    if len(df[df["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")

거래소코드가 2개인 기업 수 22 개


In [71]:
list_overlap_code

['(주)본느',
 '(주)세화피앤씨',
 '(주)아이엘사이언스',
 '(주)알로이스',
 '(주)에스에이티',
 '(주)엔케이맥스',
 '(주)원바이오젠',
 '(주)인산가',
 '(주)자비스',
 '(주)정다운',
 '(주)카이노스메드',
 '(주)티에스트릴리온',
 '(주)포인트엔지니어링',
 '(주)한국비엔씨',
 '(주)휴럼',
 '나무기술(주)',
 '소프트캠프(주)',
 '씨아이에스(주)',
 '에스케이오션플랜트(주)',
 '엠에프엠코리아(주)',
 '줌인터넷(주)',
 '패션플랫폼(주)']

In [72]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df[df["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df[df["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

In [73]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(df["거래소코드"].unique()):
    if len(df[df["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

In [74]:
## 연결 결측치 개별로 채우기
for i in range(df.shape[0]):
    if pd.isnull(df.loc[i,'유보율(IFRS연결)(백만원)']):
       df.loc[i,'유보율(IFRS연결)(백만원)'] = df.loc[i,'유보율(IFRS)']
    if pd.isnull(df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'자산(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'부채(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]자본(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'당기순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융원가(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융원가(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융원가(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융수익(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융수익(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융수익(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)'] = df.loc[i,'매출액(수익)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'연구개발비(IFRS연결)(천원)']):
       df.loc[i,'연구개발비(IFRS연결)(천원)'] = df.loc[i,'연구개발비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      유형자산(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자본금회전률(IFRS연결)']):
       df.loc[i,'자본금회전률(IFRS연결)'] = df.loc[i,'자본금회전률(IFRS)']
    if pd.isnull(df.loc[i,'총자본회전률(IFRS연결)']):
       df.loc[i,'총자본회전률(IFRS연결)'] = df.loc[i,'총자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'재고자산회전률(IFRS연결)']):
       df.loc[i,'재고자산회전률(IFRS연결)'] = df.loc[i,'재고자산회전률(IFRS)']
    if pd.isnull(df.loc[i,'유형자산회전율(IFRS연결)']):
       df.loc[i,'유형자산회전율(IFRS연결)'] = df.loc[i,'유형자산회전율(IFRS)']
    if pd.isnull(df.loc[i,'자기자본회전률(IFRS연결)']):
       df.loc[i,'자기자본회전률(IFRS연결)'] = df.loc[i,'자기자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'타인자본회전률(IFRS연결)']):
       df.loc[i,'타인자본회전률(IFRS연결)'] = df.loc[i,'타인자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'경영자본순이익률(IFRS연결)']):
       df.loc[i,'경영자본순이익률(IFRS연결)'] = df.loc[i,'경영자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'금융비용부담률(IFRS연결)']):
       df.loc[i,'금융비용부담률(IFRS연결)'] = df.loc[i,'금융비용부담률(IFRS)']
    if pd.isnull(df.loc[i,'총자본순이익률(IFRS연결)']):
       df.loc[i,'총자본순이익률(IFRS연결)'] = df.loc[i,'총자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'매출액총이익률(IFRS연결)']):
       df.loc[i,'매출액총이익률(IFRS연결)'] = df.loc[i,'매출액총이익률(IFRS)']
    if pd.isnull(df.loc[i,'자기자본순이익률(IFRS연결)']):
       df.loc[i,'자기자본순이익률(IFRS연결)'] = df.loc[i,'자기자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'총자본정상영업이익률(IFRS연결)']):
       df.loc[i,'총자본정상영업이익률(IFRS연결)'] = df.loc[i,'총자본정상영업이익률(IFRS)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)'] = df.loc[i,'자산(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'자본(*)(요약)(IFRS연결)']):
       df.loc[i,'자본(*)(요약)(IFRS연결)'] = df.loc[i,'자본(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'현금비율(IFRS연결)(백만원)']):
       df.loc[i,'현금비율(IFRS연결)(백만원)'] = df.loc[i,'현금비율(IFRS)']
    if pd.isnull(df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'차입금의존도(IFRS연결)']):
       df.loc[i,'차입금의존도(IFRS연결)'] = df.loc[i,'차입금의존도(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'감가상각비(IFRS연결)(천원)']):
       df.loc[i,'감가상각비(IFRS연결)(천원)'] = df.loc[i,'감가상각비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'부채비율(IFRS연결)']):
       df.loc[i,'부채비율(IFRS연결)'] = df.loc[i,'부채비율(IFRS)']
    if pd.isnull(df.loc[i,'당좌비율(IFRS연결)']):
       df.loc[i,'당좌비율(IFRS연결)'] = df.loc[i,'당좌비율(IFRS)']
    if pd.isnull(df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)']):
       df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)'] = df.loc[i,'*이자비용(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'순운전자본비율(IFRS연결)']):
       df.loc[i,'순운전자본비율(IFRS연결)'] = df.loc[i,'순운전자본비율(IFRS)']
    if pd.isnull(df.loc[i,'유동성장기부채(요약)(IFRS연결)(백만원)']):
       df.loc[i,'유동성장기부채(요약)(IFRS연결)(백만원)'] = df.loc[i,'유동성장기부채(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)']):
       df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)'] = df.loc[i,'단기차입금(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'유형자산의 증가(*)(IFRS연결)(천원)']):
       df.loc[i,'유형자산의 증가(*)(IFRS연결)(천원)'] = df.loc[i,'유형자산의 증가(*)(IFRS)(천원)']


In [75]:
drop_col = ['유보율(IFRS)' ,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)','[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)',
'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS)(백만원)', 
'부채(*)(요약)(IFRS)(백만원)','[제조]자본(*)(요약)(IFRS)(백만원)','당기순이익(손실)(요약)(IFRS)(백만원)', 
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)','금융원가(요약)(IFRS)(백만원)','금융수익(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)','연구개발비(IFRS)(천원)',
'[제조]      유형자산(요약)(IFRS)(백만원)','자본금회전률(IFRS)',
'총자본회전률(IFRS)','재고자산회전률(IFRS)','유형자산회전율(IFRS)','자기자본회전률(IFRS)','타인자본회전률(IFRS)',
'경영자본순이익률(IFRS)','금융비용부담률(IFRS)','총자본순이익률(IFRS)','매출액총이익률(IFRS)',
'자기자본순이익률(IFRS)','총자본정상영업이익률(IFRS)','자산(*)(요약)(IFRS)','자본(*)(요약)(IFRS)',
 '현금비율(IFRS)','* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)',
 '차입금의존도(IFRS)(백만원)','감가상각비(IFRS)(천원)','부채비율(IFRS)',
'당좌비율(IFRS)','*이자비용(요약)(IFRS)(백만원)','순운전자본비율(IFRS)',
'[제조]자산(*)(요약)(IFRS)(백만원)_x','[제조]자산(*)(요약)(IFRS연결)(백만원)_x',
'[제조]자산(*)(요약)(IFRS)(백만원)_y','[제조]자산(*)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_y', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_x', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_x',
'유동성장기부채(요약)(IFRS)(백만원)','단기차입금(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS연결)(백만원)','[제조]자본(*)(요약)(IFRS연결)(백만원)',"유형자산의 증가(*)(IFRS)(천원)"
]

In [76]:
df.drop(drop_col,axis=1,inplace=True)

In [77]:
len(df.columns)

48

In [78]:
## 성장성 지표인 증가율 컬럼 만들기
성장성_col = ['[제조]      유형자산(요약)(IFRS연결)(백만원)','매출액(수익)(요약)(IFRS연결)(백만원)',
           '자본(*)(요약)(IFRS연결)','자산(*)(요약)(IFRS연결)',
           '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']

In [79]:
## 증가율 컬럼 전기에 음수일때 당기에 음수지만 증가할때 증가율 계산
## ex (2011년 -20, 2012년 -10) -> -10-(-20) / np.abs(-20)= 
증가율_df = (df.groupby(['회사명', '거래소코드'])[성장성_col].shift(0) - df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1))*100
증가율_df.rename(columns={'[제조]      유형자산(요약)(IFRS연결)(백만원)':'유형자산증가율', '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액증가율',
       '자본(*)(요약)(IFRS연결)':'자기자본증가율', '자산(*)(요약)(IFRS연결)':'총자본증가율',
       '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'정상영업이익증가율'},inplace=True)

df.drop(['[제조]      유형자산(요약)(IFRS연결)(백만원)','[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'],axis=1,inplace=True)
df = pd.concat([df,증가율_df],axis=1)

In [80]:
## 시가총액 merge
시가총액['회계년도'] = 시가총액['회계년도'].astype(str).str[:4].astype(int)
df = df.merge(시가총액,on=['거래소코드','회계년도'],how='left')
df['시가총액(백만원)'] = df['시가총액'] / 1000000
df.drop(['시가총액'],axis=1,inplace=True)

## 거시변수 merge
거시변수.rename(columns={'회계연도':'회계년도'},inplace=True)
df = df.merge(거시변수,on=['회계년도'],how='left')

In [81]:
## 기업별 업력 설립일 이후 지속된 기간
df['업력'] = df['회계년도'] -  pd.to_datetime(df['설립일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df.drop('설립일',axis=1,inplace=True)

## 누적수익성비율  - 자산0이면 inf - 공시안함
df['누적수익성비율'] = df['[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']/df['자산(*)(요약)(IFRS연결)']

## 이자부담율
df['이자부담률'] = df['EBITDA(IFRS)(백만원)'] / df['*이자비용(요약)(IFRS연결)(백만원)'] 

## 부채상환계수
df['부채상환계수'] = df['EBITDA(IFRS)(백만원)'] / (df['*이자비용(요약)(IFRS연결)(백만원)']+df['단기차입금(요약)(IFRS연결)(백만원)'] + df['유동성장기부채(요약)(IFRS연결)(백만원)'])

## 현금흐름_자산 = 영업현금흐름 / 당기총자산
df['현금흐름 대 자산'] = df.groupby(['회사명','거래소코드'])['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'].shift(0) / df.groupby(['회사명','거래소코드'])['자산(*)(요약)(IFRS연결)'].shift(0)

## 현금흐름_매출액
df['현금흐름 대 매출액'] = df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

# 토빈Q
df['토빈Q'] = (df['부채(*)(요약)(IFRS연결)(백만원)'] + df['시가총액(백만원)']) / df['자산(*)(요약)(IFRS연결)']

## TMD
df['TMD'] = df['부채(*)(요약)(IFRS연결)(백만원)'] / df['시가총액(백만원)']

## 알트만 변수
df['시총/총자산'] = df['시가총액(백만원)'] / df['자산(*)(요약)(IFRS연결)']

## RD -단위도 수정
df['연구개발비(IFRS연결)(백만원)'] = df['연구개발비(IFRS연결)(천원)']/1000
df.drop(['연구개발비(IFRS연결)(천원)'],axis=1,inplace=True)
df['RD'] = df['연구개발비(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

## 직원수변동률 , 직원인건비총액(백만원단위)
df['직원수변동률%'] = df.groupby(['회사명','거래소코드'])['합계_기말인원(명)'].pct_change()*100
df['직원인건비총액(백만원)'] = (df['합계_년간총급여(천원)']/1000) * df['합계_기말인원(명)']
df.drop(['합계_년간총급여(천원)'],axis=1,inplace=True)

## 알트만 변수  EBIT = 법인세비용차감전순이익+금융원가-금융수익
df['EBIT/총자산'] = (df['법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] + df['금융원가(요약)(IFRS연결)(백만원)']-df['금융수익(요약)(IFRS연결)(백만원)'])/df['자산(*)(요약)(IFRS연결)']

## PBR , PER
df['PER'] = df['시가총액(백만원)'] / df['당기순이익(손실)(요약)(IFRS연결)(백만원)']
df['PBR'] = df['시가총액(백만원)']  / df['자본(*)(요약)(IFRS연결)']


# 영업손익 전기영업손익 + 당기영업손익
df["전기당기영업손익"] = (df.groupby(["회사명", "거래소코드"])["* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)"].shift(0) + df.groupby(["회사명", "거래소코드"])["* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)"].shift(1)) / 2

# capex컬럼 단위 맞춰주기
df["유형자산의 증가(*)(IFRS연결)(천원)"] = df["유형자산의 증가(*)(IFRS연결)(천원)"] / 1000

# findep =  (capex - 영업현금흐름 ) / capex
df["FINDEF"] = df["유형자산의 증가(*)(IFRS연결)(천원)"] - df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']

In [82]:
# #  연결 재무제표를 개별제무제표로 대체
# 위의 코드 요약해서 다시 세워본 코드 


# connected_cols = [col for col in 수안활.columns if "연결" in col]

# for connected_col in connected_cols:
#     original_col = connected_col.replace("연결", "")
#     if original_col in 수안활.columns:
#         수안활.loc[수안활[connected_col].isnull(), connected_col] = 수안활[original_col]

In [83]:
len(df.columns)

69

In [84]:
rename_list = {'자산(*)(요약)(IFRS연결)':'자산', '자본(*)(요약)(IFRS연결)':'자본',
       '차입금의존도(IFRS연결)':'차입금의존도', '유보율(IFRS연결)(백만원)':'유보율', '현금비율(IFRS연결)(백만원)':'현금비율',
       '부채비율(IFRS연결)':'부채비율', '당좌비율(IFRS연결)':'당좌비율', '순운전자본비율(IFRS연결)':'순운전자본비율',
       '*이자비용(요약)(IFRS연결)(백만원)':'이자비용', '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'영업손익',
       '감가상각비(IFRS연결)(천원)':'감가상각비', 'EBITDA(IFRS)(백만원)':'EBITDA', '총자본정상영업이익률(IFRS연결)':'총자본정상영업이익률',
       '자기자본순이익률(IFRS연결)':'자기자본순이익률', '총자본순이익률(IFRS연결)':'총자본순이익률', '매출액총이익률(IFRS연결)':'매출액총이익률',
       '금융비용부담률(IFRS연결)':'금융비용부담률', '경영자본순이익률(IFRS연결)':'경영자본순이익률', '자본금회전률(IFRS연결)':'자본금회전률',
       '유형자산회전율(IFRS연결)':'유형자산회전율', '재고자산회전률(IFRS연결)':'재고자산회전률', '자기자본회전률(IFRS연결)':'자기자본회전률',
       '타인자본회전률(IFRS연결)':'타인자본회전률', '총자본회전률(IFRS연결)':'총자본회전률', 
       '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)':'이익잉여금',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)':'영업활동으로 인한 현금흐름',
       '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액', '금융원가(요약)(IFRS연결)(백만원)':'금융원가',
       '금융수익(요약)(IFRS연결)(백만원)':'금융수익', '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)':'법인세비용차감전순이익',
       '당기순이익(손실)(요약)(IFRS연결)(백만원)':'당기순이익', 
       '부채(*)(요약)(IFRS연결)(백만원)':'부채', '외국인_주식분포비율':'외국인지분율', '보통주식비율':'대주주지분율',
       '단기차입금(요약)(IFRS연결)(백만원)':'단기차입금', '유동성장기부채(요약)(IFRS연결)(백만원)':'유동성장기부채',
       '시가총액(백만원)':'시가총액','연구개발비(IFRS연결)(백만원)':'연구개발비','직원인건비총액(백만원)':'직원인건비총액','통계청 한국표준산업분류 10차(대분류)':'산업군',
       "유형자산의 증가(*)(IFRS연결)(천원)" : "CAPEX"
       }

df.rename(columns=rename_list,inplace=True)

In [85]:
## 회계년도 2011년도 제거
df['상장일'] = pd.to_datetime(df['상장일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df = df[df['회계년도']>2011].reset_index(drop=True)


In [86]:
df.isna().sum()

회사명                  0
거래소코드                0
회계년도                 0
자산                1456
자본                1456
차입금의존도            1641
유보율               1641
현금비율              1641
부채비율              1641
당좌비율              1641
순운전자본비율           1641
이자비용              1456
영업손익              1456
감가상각비             1456
EBITDA            1643
총자본정상영업이익률        1641
자기자본순이익률          1641
총자본순이익률           1641
매출액총이익률           1641
금융비용부담률           1641
경영자본순이익률          1641
자본금회전률            1641
유형자산회전율           1641
재고자산회전률           1641
자기자본회전률           1641
타인자본회전률           1641
총자본회전률            1641
이익잉여금             1456
영업활동으로 인한 현금흐름    1456
매출액               1456
금융원가              1456
금융수익              1456
법인세비용차감전순이익       1456
당기순이익             1456
부채                1456
합계_기말인원(명)        2234
외국인지분율            2583
대주주지분율            2147
단기차입금             1456
유동성장기부채           1456
상장일                  0
산업군                  0
CAPEX             1456
유형자산증가율    

In [87]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자산', '자본', '차입금의존도', '유보율', '현금비율', '부채비율',
       '당좌비율', '순운전자본비율', '이자비용', '영업손익', '감가상각비', 'EBITDA', '총자본정상영업이익률',
       '자기자본순이익률', '총자본순이익률', '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률',
       '유형자산회전율', '재고자산회전률', '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금',
       '영업활동으로 인한 현금흐름', '매출액', '금융원가', '금융수익', '법인세비용차감전순이익', '당기순이익', '부채',
       '합계_기말인원(명)', '외국인지분율', '대주주지분율', '단기차입금', '유동성장기부채', '상장일', '산업군',
       'CAPEX', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율', '정상영업이익증가율', '시가총액',
       '장단기금리차', 'cpi', '업력', '누적수익성비율', '이자부담률', '부채상환계수', '현금흐름 대 자산',
       '현금흐름 대 매출액', '토빈Q', 'TMD', '시총/총자산', '연구개발비', 'RD', '직원수변동률%',
       '직원인건비총액', 'EBIT/총자산', 'PER', 'PBR', '전기당기영업손익', 'FINDEF'],
      dtype='object')

-----

-----

In [89]:
부실 = pd.read_csv('./datasets/y/y값최종.csv',index_col=0)
부실['회계년도'] = 부실['회계년도'].astype(int)
부실['거래소코드'] = 부실['거래소코드'].map(padding)
부실.drop(['번호','회사명'],axis=1,inplace=True)

In [90]:
df = df.merge(부실,on=['회계년도','거래소코드'],how='left')

In [91]:

df["공시제목"].fillna(0,inplace=True)

ind_1 = df[df["공시제목"] != 0].index

### set1
df.loc[ind_1,"target_1"] = 1

for i in df.index:
    if (df.loc[i,"target_1"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) :
        df.loc[i-1, "target_1"] = 1
df['target_1'].fillna(0,inplace=True)
df['target_1'] = df['target_1'].astype(int)

# ### set2
df.loc[ind_1,"target_2"] = 1

for i in df.index:
    if (df.loc[i,"target_2"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업') :
        df.loc[i-1, "target_2"] = 1 
    if (df.loc[i,"target_2"] == 1) and (df.loc[i-1,"회사명"] == df.loc[i-2,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-2, "target_2"] = 1

df['target_2'].fillna(0,inplace=True)
df['target_2'] = df['target_2'].astype(int)

## set_3

df.loc[ind_1,"target_3"] = 1

for i in df.index:
    if (df.loc[i,"target_3"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-1, "target_3"] = 1
df['target_3'].fillna(0,inplace=True)
df['target_3'] = df['target_3'].astype(int)


In [92]:
print(df.target_1.sum())
print(df.target_2.sum())
print(df.target_3.sum())

312
389
275


In [93]:
def get_index_with_50_or_more_nan_columns(df):
    """
    데이터프레임에서 결측치인 컬럼의 개수가 50개 이상인 인덱스 번호를 출력하는 함수
    :param df: 데이터프레임
    :return: 결측치 컬럼 개수가 50개 이상인 인덱스 번호 리스트
    """
    # 결측치 컬럼 개수 계산
    nan_column_counts = df.isnull().sum(axis=1)
    
    # 결측치 컬럼 개수가 50개 이상인 인덱스 번호 추출
    index_with_50_or_more_nan_columns = nan_column_counts[nan_column_counts >= 50].index.to_list()
    print('제거행 수:' , len(index_with_50_or_more_nan_columns))

    df.drop(index=index_with_50_or_more_nan_columns,inplace=True)
    return df


In [94]:
len(get_index_with_50_or_more_nan_columns(df))

제거행 수: 1455


12622

In [95]:
print('부실데이터 수:',df[df['target_1']==1].shape[0])

부실데이터 수: 279


In [96]:

df.isna().sum()

회사명                  0
거래소코드                0
회계년도                 0
자산                   1
자본                   1
차입금의존도             186
유보율                186
현금비율               186
부채비율               186
당좌비율               186
순운전자본비율            186
이자비용                 1
영업손익                 1
감가상각비                1
EBITDA             188
총자본정상영업이익률         186
자기자본순이익률           186
총자본순이익률            186
매출액총이익률            186
금융비용부담률            186
경영자본순이익률           186
자본금회전률             186
유형자산회전율            186
재고자산회전률            186
자기자본회전률            186
타인자본회전률            186
총자본회전률             186
이익잉여금                1
영업활동으로 인한 현금흐름       1
매출액                  1
금융원가                 1
금융수익                 1
법인세비용차감전순이익          1
당기순이익                1
부채                   1
합계_기말인원(명)        1085
외국인지분율            1351
대주주지분율            1095
단기차입금                1
유동성장기부채              1
상장일                  0
산업군                  0
CAPEX                1
유형자산증가율    

In [97]:
# 상장일이 회계년도 보다 작은 데이터 삭제
df = df[df["회계년도"] > df["상장일"]]
df.reset_index(drop = True, inplace = True)

In [98]:
# 유형자산을 모두 처분하여 감가상각비가 없어 EBITDA값이 결측치인 14년도 영진코퍼레이션 결측치에 영업이익값으로 대체

df.loc[4173, "EBITDA"] = -3063276951

# 이자부당률이 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

df.loc[4173, "이자부담률"] = -5.995505908521291

# 부채상환계수rk 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

df.loc[4173, "부채상환계수"] = -760.5303077054485

# 대주주 지분율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[903, "대주주지분율"] = 50                    # 드림티엔터테인먼트
df.loc[2913, "대주주지분율"] = 27.83                # 아라온테크
df.loc[3668, "대주주지분율"] = 49.8                 # 에이루트
df.loc[3702, "대주주지분율"] = 18.5                 # 에이제이에스
df.loc[4399, "대주주지분율"] = 23.84                # 와이즈파워
df.loc[5542, "대주주지분율"] = 100                  # 제이테크놀로지
df.loc[8155, "대주주지분율"] = 12.03                # 대한그린파워
df.loc[8326, "대주주지분율"] = 14.94                # 디브이에스코리아
df.loc[8807, "대주주지분율"] = 40.78                # 세미텍


#  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[1879, "유형자산증가율"] = 0                    # 비츠로시스
df.loc[1880, "유형자산증가율"] = 0                    # 비츠로시스

# 외국인 지분율 결측치 0으로 대체
df["외국인지분율"].fillna(0, inplace=True)


# 직원인건비총액 결측치 0으로 대체
df["직원인건비총액"].fillna(0, inplace=True)

# 합계_기말인원(명) 결측치 0으로 대체
df["합계_기말인원(명)"].fillna(0, inplace=True)



#  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 
df.loc[1879, "유형자산증가율"] = 0                    # # 유형자산을 모두 처분하여 감가상각비가 없어 EBITDA값이 결측치인 14년도 영진코퍼레이션 결측치에 영업이익값으로 대체

df.loc[4173, "EBITDA"] = -3063276951

# 이자부당률이 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

df.loc[4173, "이자부담률"] = -5.995505908521291

# 부채상환계수rk 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

df.loc[4173, "부채상환계수"] = -760.5303077054485

# 대주주 지분율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[903, "대주주지분율"] = 50                    # 드림티엔터테인먼트
df.loc[2913, "대주주지분율"] = 27.83                # 아라온테크
df.loc[3668, "대주주지분율"] = 49.8                 # 에이루트
df.loc[3702, "대주주지분율"] = 18.5                 # 에이제이에스
df.loc[4399, "대주주지분율"] = 23.84                # 와이즈파워
df.loc[5542, "대주주지분율"] = 100                  # 제이테크놀로지
df.loc[8155, "대주주지분율"] = 12.03                # 대한그린파워
df.loc[8326, "대주주지분율"] = 14.94                # 디브이에스코리아
df.loc[8807, "대주주지분율"] = 40.78                # 세미텍


#  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[1879, "유형자산증가율"] = 0                    # 비츠로시스
df.loc[1880, "유형자산증가율"] = 0                    # 비츠로시스

# 외국인 지분율 결측치 0으로 대체
df["외국인지분율"].fillna(0, inplace=True)


# 직원인건비총액 결측치 0으로 대체
df["직원인건비총액"].fillna(0, inplace=True)

# 합계_기말인원(명) 결측치 0으로 대체
df["합계_기말인원(명)"].fillna(0, inplace=True)



# #유형자산증가율 없는 컬럼 삭제

de_ind_1 = df[df[["매출액증가율"]].isna().any(1)].index
df.drop(index=de_ind_1, axis=0, inplace=True)
 # 메드팩토 셀리드 큐리언트


 # 증가율 컬럼 당해 회계년도에 상장하여 전년도 자료가 없어 결측치로 뜨는 데이터 삭제   
de_ind = df[df[["총자본증가율"]].isna().any(1)].index
df.drop(index=de_ind, axis=0, inplace=True)

# 직원수변동률 결측치 0으로 대체 
de_ind_2 = df[df[["직원수변동률%"]].isna().any(1)].index
df.drop(index=de_ind_2, axis=0, inplace=True)
#  비덴트 썬테크놀로지스

In [99]:
df.isna().sum()

회사명               0
거래소코드             0
회계년도              0
자산                0
자본                0
차입금의존도            0
유보율               0
현금비율              0
부채비율              0
당좌비율              0
순운전자본비율           0
이자비용              0
영업손익              0
감가상각비             0
EBITDA            0
총자본정상영업이익률        0
자기자본순이익률          0
총자본순이익률           0
매출액총이익률           0
금융비용부담률           0
경영자본순이익률          0
자본금회전률            0
유형자산회전율           0
재고자산회전률           0
자기자본회전률           0
타인자본회전률           0
총자본회전률            0
이익잉여금             0
영업활동으로 인한 현금흐름    0
매출액               0
금융원가              0
금융수익              0
법인세비용차감전순이익       0
당기순이익             0
부채                0
합계_기말인원(명)        0
외국인지분율            0
대주주지분율            0
단기차입금             0
유동성장기부채           0
상장일               0
산업군               0
CAPEX             0
유형자산증가율           0
매출액증가율            0
자기자본증가율           0
총자본증가율            0
정상영업이익증가율         0
시가총액              0
장단기금리차            0


---
## 결측치 처리 완료

In [100]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',None)

In [101]:
print(df[df[["이자부담률"]].isin([np.inf]).any(1)]["target_2"].value_counts())
print(df[df[["이자부담률"]].isin([-np.inf]).any(1)]["target_2"].value_counts())

0    551
1      1
Name: target_2, dtype: int64
0    95
1     4
Name: target_2, dtype: int64


In [102]:
# 이자부담률 무한대 값 삭제

a = df[df["이자부담률"].isin([np.inf])].index.to_list()
b = df[df["이자부담률"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [103]:
# 유형자산증가율 무한대 값 삭제

a = df[df["유형자산증가율"].isin([np.inf])].index.to_list()
b = df[df["유형자산증가율"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [104]:
# 현금흐름 대 매출액 무한대 값 삭제

a = df[df["현금흐름 대 매출액"].isin([np.inf])].index.to_list()
b = df[df["현금흐름 대 매출액"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [105]:
# 매출액증가율 무한대 값 삭제

a = df[df["매출액증가율"].isin([np.inf])].index.to_list()
b = df[df["매출액증가율"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

---
## 무한대 값 처리 완료

In [106]:
len(df["회사명"].unique())

1444

In [107]:
print(len(df[(df["회계년도"] < 2019) & (df["target_2"] == 1)]["회사명"].unique()))
print(len(df[(df["회계년도"] >= 2019) & (df["target_2"] == 1)]["회사명"].unique()))

123
69


---
## 산업군이 "금융 및 보험업"인 데이터 삭제

In [108]:
df[df["산업군"].str.contains("금융")][["회사명", "회계년도", "target_2", "산업군"]]

,회사명,회계년도,target_2,산업군
7924,글로벌텍스프리(주),2018,0,금융 및 보험업
7925,글로벌텍스프리(주),2019,0,금융 및 보험업
7926,글로벌텍스프리(주),2020,0,금융 및 보험업
7927,글로벌텍스프리(주),2021,0,금융 및 보험업
10699,해성산업(주),2018,0,금융 및 보험업
10700,해성산업(주),2019,0,금융 및 보험업
10701,해성산업(주),2020,0,금융 및 보험업
10702,해성산업(주),2021,0,금융 및 보험업


In [109]:
del_col = df[df["산업군"].str.contains("금융")][["회사명", "회계년도", "target_2", "산업군"]].index

df.drop(index=del_col, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)


---
# 원본 데이터 결측치 처리 완료

In [110]:
df["CAPEX"].describe()

count     10020.000000
mean      10479.159517
std       28990.891408
min        -127.980000
25%         700.176000
50%        2661.736000
75%        9431.939000
max      678148.345000
Name: CAPEX, dtype: float64

In [111]:
print(len(df[df.isin([np.inf]).any(1)].index))
print(len(df[df.isin([-np.inf]).any(1)].index))
ls_p = df[df.isin([np.inf]).any(1)].index.to_list()
ls_m = df[df.isin([-np.inf]).any(1)].index.to_list()
a = ls_p + ls_m
print(len(a))

0
0
0


In [112]:
df.loc[a, ["회사명", "회계년도"]]

,회사명,회계년도


In [113]:
pd.set_option("display.max.rows",None)
df.loc[a,:]

,회사명,거래소코드,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,이자비용,영업손익,감가상각비,EBITDA,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,매출액,금융원가,금융수익,법인세비용차감전순이익,당기순이익,부채,합계_기말인원(명),외국인지분율,대주주지분율,단기차입금,유동성장기부채,상장일,산업군,CAPEX,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,시가총액,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PER,PBR,전기당기영업손익,FINDEF,공시제목,target_1,target_2,target_3


In [114]:
df.describe()

,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,이자비용,영업손익,감가상각비,EBITDA,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,매출액,금융원가,금융수익,법인세비용차감전순이익,당기순이익,부채,합계_기말인원(명),외국인지분율,대주주지분율,단기차입금,유동성장기부채,상장일,CAPEX,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,시가총액,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PER,PBR,전기당기영업손익,FINDEF,target_1,target_2,target_3
count,10020.000000,1.002000e+04,1.002000e+04,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,1.002000e+04,1.002000e+04,1.002000e+04,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,1.002000e+04,1.002000e+04,1.002000e+04,10020.000000,10020.000000,1.002000e+04,1.002000e+04,1.002000e+04,10020.000000,10020.000000,10020.000000,1.002000e+04,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,1.002000e+04,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,10020.000000,1.002000e+04,10020.000000,10020.000000,10020.000000,1.002000e+04,1.002000e+04,10020.000000,10020.000000,10020.000000
mean,2016.927046,2.281020e+05,1.134264e+05,22.465145,1178.793615,68.809243,123.378250,223.444839,19.138910,2106.631836,8.717949e+03,1.268410e+06,-2.949070e+05,1.865214,-7.233117,-1.254605,21.949421,2.386686,-1.977968,19.992794,13.208758,211.661378,1.737998,2.492840,0.834582,4.620717e+04,1.007554e+04,1.759777e+05,4406.630739,2862.106287,6.324848e+03,4.260796e+03,1.146756e+05,248.994711,4.771194,36.163902,2.790781e+04,6798.873952,2005.393912,10479.159517,76.631212,28.958264,13.938454,11.970443,-133.202192,1.967447e+05,1.013352,97.314585,23.328543,0.081325,183.714236,124.809364,0.031295,-0.229526,1.660645,0.960948,1.236712,2682.149552,0.214520,4.730557,8.049391e+06,-0.002825,21.413692,2.310353,8.412438e+03,4.036179e+02,0.026946,0.032934,0.023752
std,2.896313,8.171924e+05,2.044788e+05,19.433904,1650.120992,145.330315,760.979153,353.437562,30.929867,4925.419991,3.369697e+04,9.020610e+06,3.060231e+07,10.441842,86.583885,16.120550,20.093282,54.891980,23.331048,35.382789,77.732121,3717.398532,1.657594,2.002982,0.529674,1.183825e+05,6.338066e+04,3.555097e+05,12789.276730,7523.293523,3.855605e+04,3.683636e+04,6.822257e+05,361.186192,8.629199,16.550049,1.958577e+05,25836.371001,6.861092,28990.891408,2975.693407,1492.619857,94.199680,41.304452,6528.330943,5.300131e+05,0.658256,3.299423,11.793430,0.797236,1861.635404,1649.790852,0.117779,8.681607,1.645348,2.228029,1.664651,6550.402627,7.547191,57.319094,4.380658e+07,0.188248,406.462013,7.927971,2.969561e+04,6.089049e+04,0.161934,0.178473,0.152285
min,2012.000000,1.629000e+03,-5.429690e+05,0.000000,-1490.240000,0.000000,0.000000,1.490000,-1243.210000,-28.000000,-1.741950e+05,-3.575100e+04,-3.063277e+09,-204.740000,-5300.390000,-318.880000,-307.410000,-705.810000,-653.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.121840e+05,-3.421598e+06,3.600000e+01,-11583.000000,0.000000,-7.659680e+05,-7.716170e+05,7.410000e+02,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1989.000000,-127.980000,-100.000000,-98.900616,-1079.677183,-95.273372,-532450.000000,1.489666e+03,0.210000,91.815000,1.000000,-38.243959,-5511.340000,-5511.340000,-3.752854,-583.029412,0.296041,0.003003,0.012305,-195.502000,-0.004381,-99.338687,0.000000e+00,-4.745856,-12184.148157,-273.924173,-1.665870e+05,-6.259930e+05,0.000000,0.000000,0.000000
25%,2014.000000,6.474100e+04,3.602675e+04,7.020000,307.590000,12.187500,33.760000,75.740000,1.727500,215.000000,-1.459500e+03,1.789422e+05,6.056725e+02,-1.650000,-8.710000,-4.410000,10.030000,-0.160000,-5.300000,4.640000,1.590000,4.220000,0.760000,1.177500,0.480000,-3.619500e+03,-1.475750e+03,3.707075e+04,508.750000,293.000

In [115]:
df.isna().sum().sum()

0

In [116]:
df.to_csv("./datasets/전처리준비/코스닥결측치처리완료.csv", encoding="utf-8-sig")